## measure coverage in text and sentences level against the reference text

In [1]:
import pandas as pd
#!python -m spacy download en_core_web_lg
from Python_Functions.userFunctions import countTfidfWord2VecScore_between_2_text
# import en_core_web_sm
# nlp=en_core_web_sm.load()

import spacy
import en_core_web_lg
nlp = en_core_web_lg.load()
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


In [2]:
# ready to model csv
Df = pd.read_csv('ready_for_model_dataset/ready_for_model.csv').drop(['Unnamed: 0'], axis = 1)
print(Df.shape)
print(Df.columns)
Df.head()

(44368, 16)
Index(['Title', 'Video_ID', 'Category', 'Age_Restricted',
       'final_corrected_version_sentences_txt', 'final_corrected_version_txt',
       'sentence_level_timstamp_min_sec', 'sentence_level_timstamp_max_sec',
       'sentence_level_timstamp_min_minute',
       'sentence_level_timstamp_max_minute', 'duration', 'Length_(min)',
       'Views_(thous)', 'main_topics', 'reference_text',
       'reference_text_cleaned'],
      dtype='object')


,Title,Video_ID,Category,Age_Restricted,final_corrected_version_sentences_txt,final_corrected_version_txt,sentence_level_timstamp_min_sec,sentence_level_timstamp_max_sec,sentence_level_timstamp_min_minute,sentence_level_timstamp_max_minute,duration,Length_(min),Views_(thous),main_topics,reference_text,reference_text_cleaned
0,Children Learn About The Ant,cXUCUvcscXs,Ant,False,welcome to my tree house come and look,welcome to my tree house come and look. at thi...,0.560,4.080,0.009333,0.068000,3.520,5.333333,921.491,insect_ant,['Ants are one of the most common insects that...,Ants are one of the most common insects that l...
1,Children Learn About The Ant,cXUCUvcscXs,Ant,False,at this,welcome to my tree house come and look. at thi...,2.560,6.080,0.042667,0.101333,3.520,5.333333,921.491,insect_ant,['Ants are one of the most common insects that...,Ants are one of the most common insects that l...
2,Children Learn About The Ant,cXUCUvcscXs,Ant,False,this is my ant farm,welcome to my tree house come and look. at thi...,4.080,8.639,0.068000,0.143983,4.559,5.333333,921.491,insect_ant,['Ants are one of the most common insects that...,Ants are one of the most common insects that l...
3,Children Learn About The Ant,cXUCUvcscXs,Ant,False,ants are amazing creatures lots of,welcome to my tree house come and look. at thi...,6.080,10.559,0.101333,0.175983,4.479,5.333333,921.491,insect_ant,['Ants are one of the most common insects that...,Ants are one of the most common insects that l...
4,Children Learn About The Ant,cXUCUvcscXs,Ant,False,insects live by themselves and have to,welcome to my tree house come and look. at thi...,8.639,12.880,0.143983,0.214667,4.241,5.333333,921.491,insect_ant,['Ants are one of the most common insects that...,Ants are one of the most common insects that l...


#### measure cosline simialrity between 2 columns

Transcript File level vs Sentences Level

- compare transcript text vs reference text
- compare transcript sentences vs reference text


- compare transcript text vs unique_vocabulary_agg(refernece text)
- compare transcript sentences vs unique_vocabulary_agg(refernece text)


- compare transcript text vs cleaned_words_agg(refernece text)
- compare transcript sentences vs cleaned_words_agg(refernece text)


In [3]:
Df_txt_lvl = Df[['Title', 'Video_ID', 'main_topics',  'final_corrected_version_txt', 'reference_text_cleaned']].drop_duplicates()
print(Df_txt_lvl.shape)
vid_ids = list(Df_txt_lvl.Video_ID.unique())
Df_sentences_lvl = Df[['Title', 'Video_ID', 'main_topics', 'final_corrected_version_sentences_txt', 'reference_text_cleaned']].drop_duplicates()
print(Df_sentences_lvl.shape)
Df_sentences_unique = Df_sentences_lvl[['Video_ID', 'final_corrected_version_sentences_txt']].drop_duplicates()
sentences = list(Df_sentences_unique.final_corrected_version_sentences_txt)
vid_ids_sent = list(Df_sentences_unique.Video_ID)
vid_sentence_combo = list(zip(vid_ids_sent,sentences))

(90, 5)
(11530, 5)


#### Based on different video id, compute the coverage score between transcript vs reference text

In [4]:
def apply_cosine_similarity(Df, new_doc_col, reference_doc_col, vid_ids = None, sentence_lvl = False, vid_sentence_combo = None):
    if sentence_lvl == False:
        result_df = pd.DataFrame()
        for Id in vid_ids:
            row_temp_unique_df = Df[Df['Video_ID'] == Id][['Video_ID', new_doc_col, reference_doc_col, 'main_topics']].drop_duplicates()
            new_doc_txt = list(row_temp_unique_df[new_doc_col])[0]
            reference_doc_txt = list(row_temp_unique_df[reference_doc_col])[0]
            try:
                coverage_scores = countTfidfWord2VecScore_between_2_text(new_doc_txt = new_doc_txt, reference_doc_txt = reference_doc_txt )
            except:
                coverage_scores = [0.0, 0.0, 0.0]
            row_temp_unique_df['Count_Vector_Cosine_Similarity'] = coverage_scores[0]
            row_temp_unique_df['TFIDF_Vector_Cosine_Similarity'] = coverage_scores[1]
            row_temp_unique_df['Word_Vector_Cosine_Similarity'] = coverage_scores[2]
            row_temp_unique_df['Avg_Cosine_Similarity'] = row_temp_unique_df[[
                'Count_Vector_Cosine_Similarity',
                'TFIDF_Vector_Cosine_Similarity',
                'Word_Vector_Cosine_Similarity']].mean(axis=1)
            result_df = pd.concat([result_df,row_temp_unique_df], axis = 0).sort_values(by = 'Avg_Cosine_Similarity', ascending = False)
        return result_df
    else:
        result_df = pd.DataFrame()
        for combo in vid_sentence_combo:
            row_temp_unique_df = Df[(Df['Video_ID'] == combo[0]) & (Df[new_doc_col] == combo[1])][['Video_ID', new_doc_col, reference_doc_col, 'main_topics']].drop_duplicates()
            new_doc_txt = list(row_temp_unique_df[new_doc_col])[0]
            reference_doc_txt = list(row_temp_unique_df[reference_doc_col])[0]
            try:
                coverage_scores = countTfidfWord2VecScore_between_2_text(new_doc_txt = new_doc_txt, reference_doc_txt = reference_doc_txt )
            except:
                coverage_scores = [0.0, 0.0, 0.0]
            row_temp_unique_df['Count_Vector_Cosine_Similarity'] = coverage_scores[0]
            row_temp_unique_df['TFIDF_Vector_Cosine_Similarity'] = coverage_scores[1]
            row_temp_unique_df['Word_Vector_Cosine_Similarity'] = coverage_scores[2]
            row_temp_unique_df['Avg_Cosine_Similarity'] = row_temp_unique_df[[
                'Count_Vector_Cosine_Similarity',
                'TFIDF_Vector_Cosine_Similarity',
                'Word_Vector_Cosine_Similarity']].mean(axis=1)
            result_df = pd.concat([result_df,row_temp_unique_df], axis = 0).sort_values(by = 'Avg_Cosine_Similarity', ascending = False)
        return result_df    

In [5]:
Df_txt_lvl_txt_vs_reference_text = apply_cosine_similarity(
    Df = Df_txt_lvl,
    vid_ids = vid_ids,
    new_doc_col = 'final_corrected_version_txt',
    reference_doc_col = 'reference_text_cleaned'
)

Df_txt_lvl_txt_vs_reference_text.head()

,Video_ID,final_corrected_version_txt,reference_text_cleaned,main_topics,Count_Vector_Cosine_Similarity,TFIDF_Vector_Cosine_Similarity,Word_Vector_Cosine_Similarity,Avg_Cosine_Similarity
1345,VVX9M6WYCjo,[Music]. ants. ants all around. do you know ho...,Ants are one of the most common insects that l...,insect_ant,0.856380,0.778180,0.974109,0.869556
696,zL7VIomiiyg,today we're going to learn all about. ants. oh...,Ants are one of the most common insects that l...,insect_ant,0.828937,0.740823,0.939397,0.836386
44332,birds_intro_flamingo_v,Let's learn about flamingos flamingos are tall...,Flamingo Flamingo is about 110 to 150 cm tall ...,bird_flamingo,0.762279,0.646343,0.955252,0.787958
22227,_gs4XfbYPKc,You're watching FreeSchool!. Let's learn about...,Flamingo Flamingo is about 110 to 150 cm tall ...,bird_flamingo,0.766463,0.644687,0.945320,0.785490
22308,FMjppU6AKBQ,did you know that there's a popular bird. name...,Flamingo Flamingo is about 110 to 150 cm tall ...,bird_flamingo,0.765567,0.665743,0.906376,0.779229


##### Insight: given 5 main topics, what are their top 5 videos transcript text based on the coverage score?

In [6]:
Df_txt_lvl_txt_vs_reference_text.to_csv('model_output_dataset/coverage_measure_output_text.csv')

main_topics_Lst = list(Df_txt_lvl_txt_vs_reference_text.main_topics.unique())

for topic in main_topics_Lst:
    print()
    print('top 10 video under main topic: {} :'.format(topic))
    temp_df = (Df_txt_lvl_txt_vs_reference_text[Df_txt_lvl_txt_vs_reference_text['main_topics'] == topic]
     .sort_values(by = 'Avg_Cosine_Similarity', ascending = False)
    )
    print()
    print(temp_df.head(10)['Video_ID'])


top 10 video under main topic: insect_ant :

1345             VVX9M6WYCjo
696              zL7VIomiiyg
874              CASrmm4BUJk
1905             9SPixBok5ls
635              2IVb2Atu3Jc
0                cXUCUvcscXs
1022             A_hEZNxG_H8
44245    insects_intro_ant_v
546              QNnmjyHPnbc
1513             NFzqL5pjjpw
Name: Video_ID, dtype: object

top 10 video under main topic: bird_flamingo :

44332    birds_intro_flamingo_v
22227               _gs4XfbYPKc
22308               FMjppU6AKBQ
24802               mWwtO5adfTE
22265               b2AN1cPn3zY
22531               Pz3XiJac57w
24722               lYDNA3XQV_0
23702               ox7FcSCk4qw
23673               QLV_K7DVeyU
24244               CGg12TCEGnE
Name: Video_ID, dtype: object

top 10 video under main topic: bird_peacock :

29623              ovxJXZdhRdM
29352              EkJvstgd7oc
44298    birds_intro_peacock_v
29494              9WW6YJen-xw
29828              hAyMnZN1zpE
33261              Nl_dbTgGpDQ
3

#### Based on different video id, compute the coverage score between sentence vs reference text

In [7]:
# new_doc_col = 'final_corrected_version_sentences_txt'
# reference_doc_col = 'reference_text_cleaned'


# result_df = pd.DataFrame()
# for combo in vid_sentence_combo:
# #     print(combo[0])
# #     print(combo[1])
#     row_temp_unique_df = Df_sentences_lvl[(Df_sentences_lvl['Video_ID'] == combo[0]) & \
#                                           (Df_sentences_lvl[new_doc_col] == combo[1])][\
#                                         ['Video_ID', new_doc_col, reference_doc_col, 'main_topics']].drop_duplicates()
# #     print(row_temp_unique_df)
#     new_doc_txt = list(row_temp_unique_df[new_doc_col])[0]
# #     print(new_doc_txt)
#     reference_doc_txt = list(row_temp_unique_df[reference_doc_col])[0]
#     try:
#         coverage_scores = countTfidfWord2VecScore_between_2_text(new_doc_txt = new_doc_txt, reference_doc_txt = reference_doc_txt )
#     except:
#         coverage_scores = [0.0, 0.0, 0.0]
#     print(coverage_scores)
#     row_temp_unique_df['Count_Vector_Cosine_Similarity'] = coverage_scores[0]
#     row_temp_unique_df['TFIDF_Vector_Cosine_Similarity'] = coverage_scores[1]
#     row_temp_unique_df['Word_Vector_Cosine_Similarity'] = coverage_scores[2]
#     row_temp_unique_df['Avg_Cosine_Similarity'] = row_temp_unique_df[[
#         'Count_Vector_Cosine_Similarity',
#         'TFIDF_Vector_Cosine_Similarity',
#         'Word_Vector_Cosine_Similarity']].mean(axis=1)
#     result_df = pd.concat([result_df,row_temp_unique_df], axis = 0).sort_values(by = 'Avg_Cosine_Similarity', ascending = False)
# result_df    



In [8]:
Df_sentences_lvl_txt_vs_reference_text = apply_cosine_similarity(
    Df = Df_sentences_lvl,
    sentence_lvl = True,
    vid_sentence_combo = vid_sentence_combo,
    new_doc_col = 'final_corrected_version_sentences_txt',
    reference_doc_col = 'reference_text_cleaned'
)

Df_sentences_lvl_txt_vs_reference_text.head()

,Video_ID,final_corrected_version_sentences_txt,reference_text_cleaned,main_topics,Count_Vector_Cosine_Similarity,TFIDF_Vector_Cosine_Similarity,Word_Vector_Cosine_Similarity,Avg_Cosine_Similarity
267,kFiDThjUBTk,the ants are,Ants are one of the most common insects that l...,insect_ant,0.842084,0.743226,0.703941,0.763084
1367,VVX9M6WYCjo,one ant two ant three and four,Ants are one of the most common insects that l...,insect_ant,0.842084,0.743226,0.703941,0.763084
591,QNnmjyHPnbc,ants,Ants are one of the most common insects that l...,insect_ant,0.842084,0.743226,0.703941,0.763084
1347,VVX9M6WYCjo,ants all around,Ants are one of the most common insects that l...,insect_ant,0.842084,0.743226,0.703941,0.763084
1346,VVX9M6WYCjo,ants,Ants are one of the most common insects that l...,insect_ant,0.842084,0.743226,0.703941,0.763084


##### Insights: Given a video, what are the top 5 sentences under that video ?

In [9]:
np.logspace(0,-9, num=50)

array([1.00000000e+00, 6.55128557e-01, 4.29193426e-01, 2.81176870e-01,
       1.84206997e-01, 1.20679264e-01, 7.90604321e-02, 5.17947468e-02,
       3.39322177e-02, 2.22299648e-02, 1.45634848e-02, 9.54095476e-03,
       6.25055193e-03, 4.09491506e-03, 2.68269580e-03, 1.75751062e-03,
       1.15139540e-03, 7.54312006e-04, 4.94171336e-04, 3.23745754e-04,
       2.12095089e-04, 1.38949549e-04, 9.10298178e-05, 5.96362332e-05,
       3.90693994e-05, 2.55954792e-05, 1.67683294e-05, 1.09854114e-05,
       7.19685673e-06, 4.71486636e-06, 3.08884360e-06, 2.02358965e-06,
       1.32571137e-06, 8.68511374e-07, 5.68986603e-07, 3.72759372e-07,
       2.44205309e-07, 1.59985872e-07, 1.04811313e-07, 6.86648845e-08,
       4.49843267e-08, 2.94705170e-08, 1.93069773e-08, 1.26485522e-08,
       8.28642773e-09, 5.42867544e-09, 3.55648031e-09, 2.32995181e-09,
       1.52641797e-09, 1.00000000e-09])

In [10]:
Df_sentences_lvl_txt_vs_reference_text.to_csv('model_output_dataset/coverage_measure_output_sentences.csv')

(Df_sentences_lvl_txt_vs_reference_text[Df_sentences_lvl_txt_vs_reference_text['Video_ID'] == 'FMjppU6AKBQ']
 .sort_values(by = 'Avg_Cosine_Similarity', ascending = False)
 .head()['final_corrected_version_sentences_txt']
)

22327                            flamingos
22371                        flamingo from
22332                        name flamingo
22373                             flamingo
22505    a flamingo i don't even have legs
Name: final_corrected_version_sentences_txt, dtype: object